In [2]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv', sep=',')
melb_df['Date'] = pd.to_datetime(melb_df['Date'])

exclude_colu = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_uniq_cnt = 150
for i_col in melb_df.columns:
    if melb_df[i_col].nunique() < (max_uniq_cnt) and i_col not in exclude_colu:
        melb_df[i_col] = melb_df[i_col].astype('category')

# Метод groupby как способ построения сводных таблиц

In [ ]:
# зависимость медианной цены и площади здания от числа комнат:

melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [ ]:
# можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). 
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

#В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов: первый уровень — число комнат, 
# второй уровень — тип здания. Такая организация индексов называется иерархической. Вычисление параметра (средней цены) происходит 
# во всех возможных комбинациях признаков.

melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

C:\Users\user\AppData\Local\Temp\ipykernel_6828\2831942256.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_df.groupby(['Rooms', 'Type'])['Price'].mean()


Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
       unit                  NaN
6      house        1.869508e+06
       townhouse             NaN
       unit         5.200000e+05
7      house        1.920700e+06
       townhouse             NaN
       unit                  NaN
8      house        1.510286e+06
       townhouse             NaN
       unit         2.250000e+06
10     house        9.000000e+05
       townhouse             NaN
       unit                  NaN
Name: Price, dtype: float64

In [11]:
# продолжение предыдущего
# Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам),
# а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(), который позволяет переопределить 
# вложенный индекс в виде столбцов таблицы:

melb_df.groupby(['Rooms', 'Type'])["Price"].mean().unstack()

C:\Users\user\AppData\Local\Temp\ipykernel_6828\2803039870.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_df.groupby(['Rooms', 'Type'])["Price"].mean().unstack()


Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


# Метод pivot_table для построения сводных таблиц

In [13]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

C:\Users\user\AppData\Local\Temp\ipykernel_6828\3628189379.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [36]:
# давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. 
# Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет 
# располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. 
# В качестве значений сводной таблицы возьмём количество продаж.

melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='mean'
).sort_values(by=1)

C:\Users\user\AppData\Local\Temp\ipykernel_6828\2327485744.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


Weekend,0,1
Regionname,,
Western Victoria,3.917500e+05,3.994479e+05
Northern Victoria,6.414545e+05,5.777333e+05
Eastern Victoria,7.759231e+05,6.752996e+05
Western Metropolitan,8.949470e+05,8.526451e+05
Northern Metropolitan,8.818808e+05,9.059573e+05
South-Eastern Metropolitan,9.181675e+05,9.247404e+05
Eastern Metropolitan,1.082900e+06,1.113325e+06
Southern Metropolitan,1.321720e+06,1.397831e+06


In [37]:
#найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). 
# Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. 
# Построим такую сводную таблицу, где пропущенные значения заменим на 0:

melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

C:\Users\user\AppData\Local\Temp\ipykernel_6828\876979591.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_6828\876979591.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


median                          mean              \
Type                        house townhouse   unit        house   townhouse   
Regionname                                                                    
Eastern Metropolitan        674.0     233.5  203.0   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230.0  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0.0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0.0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199.0   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0.0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62.0   507.883406  244.560669   
Western Victoria            599.5       0.0    0.0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

## Многомерные сводные таблицы

In [38]:
# построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), 
# по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

melb_df.pivot_table(
    values='Price',
    index=['Method', 'Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

C:\Users\user\AppData\Local\Temp\ipykernel_6828\2133305943.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                 1244000.0          780000.0   
       townhouse              760000.0               0.0   
       unit                   650000.0               0.0   
S      house                 1127000.0          675000.0   
       townhouse              828000.0               0.0   
       unit                   645750.0          492000.0   
SA     house                  932500.0          950000.0   
       townhouse              807500.0               0.0   
       unit                        0.0               0.0   
SP     house                 1050000.0          672500.0   
       townhouse              910000.0               0.0   
       unit                   515000.0          400000.0   
VB     house                 1100000.0          712500.0   
       townhouse              892500.0               0.0   
       unit                   500000.0               0.0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                   900000.0           500000.0   
       townhouse               632500.0                0.0   
       unit                    410000.0                0.0   
S      house                   920000.0           555000.0   
       townhouse               750000.0                0.0   
       unit                    525500.0                0.0   
SA     house                   817500.0           540000.0   
       townhouse               425000.0                0.0   
       unit                    616000.0                0.0   
SP     house                   900000.0           521000.0   
       townhouse               690000.0                0.0   
       unit                    470000.0                0.0   
VB     house                  1050000.0           690000.0   
       townhouse               640000.0                0.0   
       unit                    450000.0                0.0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                        865000.0              1725000.0   
       townhouse                   1190000.0              1055000.0   
       unit                         525000.0               571250.0   
S      house                        883300.0              1611000.0   
       townhouse                    875000.0              1135000.0   
       unit                         606000.0               655000.0   
SA     house                        880000.0              1390000.0   
       townhouse                         0.0              1141000.0   
       unit                              0.0               580000.0   
SP     house                        770000.0              1521750.0   
       townhouse                    800000.0              1162500.0   
       unit                         601000.0               550000.0   
VB     house                        850000.0              1800000.0   
       townhouse                         0.0              1250000.0   
       unit                         700000.0               500000.0   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                  870000.0          630000.0  
       townhouse              670000.0               0.0  
       unit                   360000.0               0.0  
S      house                  870000.0          397500.0  
       townhouse              729000.0               0.0  
       unit                   489000.0               0.0  
SA     house                  772500.0               0.0  
       townhouse              467500.0               0.0  
       unit                   571000.0               0.0  
SP     house                  865000.0          360000.0  
       t

## Доступ к данным в сводной таблице

In [ ]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
    )

pivot.columns #отображение се столбцов табицы pivot

C:\Users\user\AppData\Local\Temp\ipykernel_6828\2502400467.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = melb_df.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_6828\2502400467.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = melb_df.pivot_table(


MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [ ]:
pivot['mean']['unit'] #получить средние значения (mean) площадей участков для типа здания unit

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [ ]:
mask = pivot['mean', 'house'] < pivot['median', 'house'] #маска для фильтрации
filt_pivod = pivot[mask] # фильтрация таблицы
filt_pivod


median                        mean              \
Type                   house townhouse  unit       house   townhouse   
Regionname                                                             
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824   
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

In [ ]:
list(filt_pivod.index) #индексы отфильтрованной свойдной таблицы

['Southern Metropolitan', 'Western Metropolitan']

In [ ]:
list(pivot.index) #индексы исходной сводной таблицы

['Eastern Metropolitan',
 'Eastern Victoria',
 'Northern Metropolitan',
 'Northern Victoria',
 'South-Eastern Metropolitan',
 'Southern Metropolitan',
 'Western Metropolitan',
 'Western Victoria']

# Синтаксис для создания в ручную мультииндексную таблицу

In [56]:
import numpy as np
mres = pd.Series(np.random.rand(8),
                 index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
mres

white  up       0.397724
       down     0.654752
       right    0.111371
blue   up       0.353235
       down     0.706963
red    up       0.255075
       down     0.203150
       left     0.918673
dtype: float64

In [57]:
mres.index

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )

In [61]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
mframe

pen               paper          
                   1         2         1         2
white up    1.778856 -0.742866 -1.848446  0.940309
      down  0.629918 -1.367807 -0.084627  2.086619
red   up   -0.455781  1.147442 -0.122664  0.792518
      down  2.334378  1.434374 -0.485359  0.329381

In [62]:
mframe.index

MultiIndex([('white',   'up'),
            ('white', 'down'),
            (  'red',   'up'),
            (  'red', 'down')],
           )

In [63]:
mframe.columns

MultiIndex([(  'pen', 1),
            (  'pen', 2),
            ('paper', 1),
            ('paper', 2)],
           )

#

In [66]:
#Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) 
# и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?

melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    fill_value=0,
    aggfunc='median'
)

C:\Users\user\AppData\Local\Temp\ipykernel_6828\384569110.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126.0,126.0,141.0,177.0,126.0,216.5,126.0,126.0
townhouse,88.0,114.0,126.0,159.5,152.0,0.0,0.0,0.0,0.0
unit,69.5,110.0,126.0,126.0,0.0,171.0,0.0,126.0,0.0


In [70]:
# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства 
# (SellerG) и типа здания (Type).

melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    fill_value=0,
    aggfunc='median'
).sort_values(by='unit', ascending=False)

C:\Users\user\AppData\Local\Temp\ipykernel_6828\3195889250.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_df.pivot_table(


Type,house,townhouse,unit
SellerG,,,
Nick,2025000.0,780000.0,900000.0
Marshall,1975000.0,1408500.0,715000.0
Cayzer,1505000.0,1450000.0,707500.0
Kay,2220000.0,1365000.0,695000.0
Noel,1400500.0,990000.0,693250.0
Buxton,1323750.0,1030000.0,670000.0
Fletchers,1390000.0,1238000.0,653000.0
Chisholm,1520000.0,950000.0,640000.0
Philip,1035000.0,701000.0,636000.0
